<a href="https://colab.research.google.com/github/SHASHANKV9611/Vehicle_count/blob/main/vehicle_count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ultralytics

In [2]:
SOURCE_VIDEO_PATH ="/content/drive/MyDrive/count/vehicles.mp4"

In [3]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('yolov8n.pt')

# Run inference on 'bus.jpg' with arguments
model.predict(source=SOURCE_VIDEO_PATH, save=True, imgsz=320, conf=0.5)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/512) /content/drive/MyDrive/count/vehicles.mp4: 192x320 4 cars, 1 truck, 168.7ms
video 1/1 (2/512) /content/drive/MyDrive/count/vehicles.mp4: 192x320 3 cars, 1 truck, 72.3ms
video 1/1 (3/512) /content/drive/MyDrive/count/vehicles.mp4: 192x320 2 cars, 72.4ms
video 1/1 (4/512) /content/drive/MyDrive/count/vehicles.mp4: 192x320 2 cars, 1 bus, 78.9ms
video 1/1 (5/512) /content/drive/MyDrive/count/vehicles.mp4: 192x320 3 cars, 137.1ms
video 1/1

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

## Installing the supervision for video sink purpose

In [4]:
pip install supervision[Desktop]

### To mark the coordinates of the frame, creating a function the download the frame.

In [5]:
import cv2

def download_frame(video_path, frame_number, output_path):
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    if ret:
        cv2.imwrite(output_path, frame)
    cap.release()

# Example usage:
video_path = '/content/drive/MyDrive/count/vehicles.mp4'
frame_number = 100  # Change this to the desired frame number
output_path = 'output_frame.jpg'  # Change this to the desired output path

download_frame(video_path, frame_number, output_path)


In [6]:
import cv2
from collections import defaultdict
import supervision as sv
from ultralytics import YOLO
from supervision import Detections

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Set up video capture
cap = cv2.VideoCapture("/content/drive/MyDrive/count/vehicles.mp4")

# Define the line coordinates
START = sv.Point(248, 438)
END = sv.Point(972, 438)


# Store the track history
track_history = defaultdict(lambda: [])

# Create a dictionary to keep track of objects that have crossed the line
crossed_objects = {}

# Open a video sink for the output video
video_info = sv.VideoInfo.from_video_path("/content/drive/MyDrive/count/vehicles.mp4")
with sv.VideoSink("output.mp4", video_info) as sink:

    while cap.isOpened():
        success, frame = cap.read()

        if success:
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, classes=[2, 3, 5, 7], persist=True, save=True, tracker="bytetrack.yaml")

            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Visualize the results on the frame
            annotated_frame = results[0].plot()


            # Plot the tracks and count objects crossing the line
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point


                # Check if the object crosses the line
                if START.x < x < END.x and abs(y - START.y) < 5:  # Assuming objects cross horizontally
                    if track_id not in crossed_objects:
                        crossed_objects[track_id] = True

                    # Annotate the object as it crosses the line
                    cv2.rectangle(annotated_frame, (int(x - w / 2), int(y - h / 2)), (int(x + w / 2), int(y + h / 2)), (0, 255, 0), 2)

            # Draw the line on the frame
            cv2.line(annotated_frame, (START.x, START.y), (END.x, END.y), (0, 255, 0), 2)

            # Write the count of objects on each frame
            count_text = f"Objects crossed: {len(crossed_objects)}"
            cv2.putText(annotated_frame, count_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Write the frame with annotations to the output video
            sink.write_frame(annotated_frame)
        else:
            break

# Release the video capture
cap.release()



0: 384x640 6 cars, 1 truck, 199.8ms
Speed: 4.3ms preprocess, 199.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict15

0: 384x640 6 cars, 1 truck, 147.6ms
Speed: 3.8ms preprocess, 147.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict15

0: 384x640 7 cars, 1 truck, 137.0ms
Speed: 2.7ms preprocess, 137.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict15

0: 384x640 6 cars, 2 trucks, 139.3ms
Speed: 2.7ms preprocess, 139.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict15

0: 384x640 7 cars, 1 truck, 126.7ms
Speed: 2.8ms preprocess, 126.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict15

0: 384x640 7 cars, 1 truck, 154.1ms
Speed: 2.9ms preprocess, 154.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384,